In [2]:
import os
import pandas as pd
from pandas.io import gbq
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import exc
from datetime import datetime
from io import StringIO
import re
import sys

In [3]:
path = 'C:\\Users\\adamb\\Desktop\\BOT\\'
lista_arquivos = os.listdir(path)

In [4]:
dici_arquivos = {
    'sales': 'Base_{}.csv'
}

In [5]:
def have_tables(dici_arquivos):
    tabelas_faltante = []
    for tabela in dici_arquivos:
        if dici_arquivos[tabela] == '':
            tabelas_faltante.append(tabela)
    
    if len(tabelas_faltante)>0:
        print('Estão faltando as seguintes tabelas:\n',tabelas_faltante)

In [6]:
def get_list_of_dataframe(arquivo):
    prefixo_arquivo = arquivo.replace('{}.csv', '')
    arquivo_diretorio = os.listdir(path)
    lista_arquivos_para_agrupar = [arquivo for arquivo in arquivo_diretorio if arquivo.startswith(prefixo_arquivo) ]
    # Agrupar arquivos em 1 único dataframe
    lista_df = []
    print('lido os arquivos: ')
    for arquivo in lista_arquivos_para_agrupar:  
        print(arquivo)
        lista_df.append(text_file_to_dataframe(path+arquivo))
    return lista_df

In [7]:
def text_file_to_dataframe(path_file):
    try:
        return pd.read_csv(path_file, sep=';', encoding='utf8', dtype='str')
    except pd.errors.ParserError as e:
        with open("temp.log", "w") as f: 
            original_stderr = sys.stderr
            sys.stderr = f
            df = pd.read_csv(path_file, sep=';', encoding='utf8', error_bad_lines=False, dtype='str')
            sys.stderr = original_stderr
            
        # Exibir as linhas que apresentaram erro    
        with open("temp.log", "r") as f: 
            lines = f.readlines()[0].split('\\n')
            errors = []
            for line in lines:
                errors += [re.findall('\d+', line)]
                errors_lines = [int(first[0]) for first in errors if len(first)>0]
                
        print('Arquivos com linhas quebradas: {} \nLinhas:{}'.format(path_file, errors_lines))
        
        
    return df

In [8]:
def read_file(tabela, path):
    arquivo = dici_arquivos[tabela]
    # verificar se a tabela está dividida em multiplos arquivo
    if '{}' in arquivo:
        lista_df = get_list_of_dataframe(arquivo)
        df = pd.concat(lista_df, sort=False)
    else:
        df = text_file_to_dataframe(path+arquivo)
        
    df.archive = arquivo
    df.table = tabela
    return df

In [9]:
'''
Passar essa responsabilidade para o read_file
'''
def read_from_csv_deleting_lines(tabela, lines=[]):
    with open(path + dici_arquivos[tabela], "r", encoding='utf8') as in_file:
        arquivo_lista = in_file.read().splitlines()
        for line in lines:
            if str(line)[0]=='>':
                line = int(line[1:])
                del arquivo_lista[line-1:]
            else: 
                del arquivo_lista[line-1]
    arquivo_texto = StringIO('\n'.join(arquivo_lista))
    df = pd.read_csv(arquivo_texto, sep=';')
    return df

In [10]:
def load_to_db(df):
    df.to_sql('sales'.format(tabela),
          con=engine,
          schema=os.getenv('SCHEMA'),
          #if_exists='replace',
          index=False,
          chunksize=10000,
          method='multi'
         )

In [11]:
have_tables(dici_arquivos)

In [12]:
tabela = 'sales'

In [13]:
df = read_file(tabela, path)

lido os arquivos: 
Base_2017_1.csv
Base_2018_2.csv
Base_2019_3.csv


In [14]:
df.head()

,ID_MARCA,MARCA,ID_LINHA,LINHA,DATA_VENDA,QTD_VENDA
0,1,BOTICÁRIO,1,CABELOS,27/03/2018,9
1,4,VULT,1,CABELOS,03/02/2018,3
2,2,EUDORA,1,CABELOS,21/10/2018,7
3,2,EUDORA,4,SOLAR,22/04/2018,13
4,5,BELEZA NA WEB,3,MAQUIAGEM,21/03/2018,9


In [15]:
df.dtypes

ID_MARCA      object
MARCA         object
ID_LINHA      object
LINHA         object
DATA_VENDA    object
QTD_VENDA     object
dtype: object

In [16]:
df.shape

(3000, 6)

In [17]:
df["ID_MARCA"]= df["ID_MARCA"].astype(int)
df["MARCA"]= df["MARCA"].astype(str)
df["ID_LINHA"]= df["ID_LINHA"].astype(int)
df["LINHA"]= df["LINHA"].astype(str)
df["QTD_VENDA"]= df["QTD_VENDA"].astype(int)
df["DATA_VENDA"]= pd.to_datetime(df["DATA_VENDA"])

In [18]:
df.dtypes

ID_MARCA               int32
MARCA                 object
ID_LINHA               int32
LINHA                 object
DATA_VENDA    datetime64[ns]
QTD_VENDA              int32
dtype: object

In [19]:
df.head()

,ID_MARCA,MARCA,ID_LINHA,LINHA,DATA_VENDA,QTD_VENDA
0,1,BOTICÁRIO,1,CABELOS,2018-03-27,9
1,4,VULT,1,CABELOS,2018-03-02,3
2,2,EUDORA,1,CABELOS,2018-10-21,7
3,2,EUDORA,4,SOLAR,2018-04-22,13
4,5,BELEZA NA WEB,3,MAQUIAGEM,2018-03-21,9


In [ ]:
df.to_gbq(destination_table='bigqueryDB.stage_table',project_id='boticario-294316',if_exists='fail')